In [18]:

import numpy as np
import torch
import torchvision.models as models
from torch import nn
from torch.utils.data import DataLoader

from sklearn.metrics.pairwise import cosine_similarity

In [19]:
test_dataset = torch.load("data/test_dataset.pt")

In [20]:
import os

baseline = models.resnet50(pretrained=True)
    
baseline.fc = nn.Linear(2048, 910)
    
if os.path.exists(f"models/baseline.pt"):
    baseline.load_state_dict(torch.load(f"models/baseline.pt"))

baseline.fc = torch.nn.Identity()

c:\Users\sebas\miniconda3\envs\trust-me\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\sebas\miniconda3\envs\trust-me\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [31]:
triplet = models.resnet18(pretrained=True)
    
    
if os.path.exists(f"models/resnet18_9.pt"):
    triplet.load_state_dict(torch.load(f"models/resnet18_9.pt"))

triplet.fc = torch.nn.Identity()

c:\Users\sebas\miniconda3\envs\trust-me\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\sebas\miniconda3\envs\trust-me\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [22]:
def get_accuracy(embeddings, targets):
    embeddings = np.array(embeddings)
    targets = np.array(targets)
    
    # Get cosine similarity for all embeddings
    similarity_matrix = cosine_similarity(embeddings)
    
    # Fill diagonal with inf
    np.fill_diagonal(similarity_matrix, -np.inf)
    
    # Find the index of max sim for each embedding
    most_similar_indices = np.argmax(similarity_matrix, axis=1)
    
    # Extract the labels of the most similar items
    predicted_labels = [targets[i] for i in most_similar_indices]
    n_correct = sum(1 for true, pred in zip(targets, predicted_labels) if true == pred)
    
    accuracy = 100 * (n_correct / len(embeddings))
    
    return accuracy, predicted_labels

In [23]:
def get_embeddings(model, dataloader, device):
    embeddings = []
    targets = []
    
    with torch.no_grad():
        for inputs, labels in dataloader:
            data = inputs.to(device)
            label = labels.to(device)
            batch_emb, batch_lab = [], []            
            
            emb = model(data)
            
            batch_emb.extend(emb.cpu())
            batch_lab.extend(label.cpu().tolist())
            
            embeddings.extend(batch_emb)
            targets.extend(batch_lab)
            
    return embeddings, targets

In [24]:
test_dataloader = DataLoader(test_dataset, 1, True)

In [25]:
device = "cuda"
baseline = baseline.to(device)

baseline_emb, baseline_targ = get_embeddings(baseline, test_dataloader, "cuda")

In [26]:
for i in range(len(baseline_emb)):
    baseline_emb[i] = baseline_emb[i].detach().numpy()

In [27]:
type(baseline_emb[0])

numpy.ndarray

In [28]:
baseline_accuracy, baseline_preds = get_accuracy(baseline_emb, baseline_targ)

In [32]:
triplet = triplet.to(device)
triplet_emb, triplet_targ = get_embeddings(triplet, test_dataloader, "cuda")

for i in range(len(triplet_emb)):
    triplet_emb[i] = triplet_emb[i].detach().numpy()

triplet_accuracy, triplet_preds = get_accuracy(triplet_emb, triplet_targ)

In [30]:
print(f"Test set with {len(test_dataset)} samples of {len(set(test_dataset.labels))} classes.")
print(f"Baseline model openset performance: {baseline_accuracy}")
print(f"Triplet model openset performance: {triplet_accuracy}")

Test set with 75576 samples of 400 classes.
Baseline model openset performance: 79.31486186090822
Triplet model openset performance: 3.810733566211496
